In [1]:
from flask import Flask, request
import requests
from config import PAGE_ACCESS_TOKEN, VERIFY_TOKEN # Assuming you will define these here

In [2]:
app = Flask(__name__)

In [3]:
# Replace these with your own tokens
PAGE_ACCESS_TOKEN = PAGE_ACCESS_TOKEN
VERIFY_TOKEN = VERIFY_TOKEN

In [4]:
# --- Dynamic Question & Answer Database ---
# You can add or remove as many entries as you need.
# The bot will match the user's message to the keys in this dictionary.
qa_database = {
    "hello": "Hello! How can I help you today?",
    "hi": "Hi there! What can I do for you?",
    "what is your purpose?": "I am a simple Messenger bot designed to answer questions from my predefined knowledge base.",
    "how do i get started?": "To get started, simply type a question I might know the answer to, like 'hello' or 'what is your purpose?'",
    "thank you": "You're welcome! I'm glad I could help."
}

In [5]:
# --- Root route for testing ---
@app.route("/", methods=["GET"])
def home():
    return "Messenger bot is running!"

In [6]:
# --- Webhook route for Facebook ---
@app.route("/webhook", methods=["GET", "POST"])
def webhook():
    if request.method == "GET":
        # Verification handshake
        token_sent = request.args.get("hub.verify_token")
        return verify_fb_token(token_sent)
    else:
        # Handle messages
        output = request.get_json()
        for event in output.get("entry", []):
            for messaging_event in event.get("messaging", []):
                sender_id = messaging_event["sender"]["id"]
                if messaging_event.get("message"):
                    message_text = messaging_event["message"].get("text")
                    if message_text:
                        # Convert message to lowercase for case-insensitive matching
                        message_text_lower = message_text.lower().strip()
                        
                        # Look for a response in the QA database
                        response_text = qa_database.get(message_text_lower)
                        
                        if response_text:
                            send_message(sender_id, response_text)
                        else:
                            # Default response if no match is found
                            default_response = "I'm sorry, I don't understand that question. You can try asking one of the following:\n"
                            # List all available questions
                            for q in qa_database.keys():
                                default_response += f"- {q}\n"
                            send_message(sender_id, default_response)
        return "Message Processed", 200

In [7]:
def verify_fb_token(token_sent):
    if token_sent == VERIFY_TOKEN:
        return request.args.get("hub.challenge")
    return "Invalid verification token"

In [8]:
def send_message(recipient_id, text):
    """Send message back to Facebook Messenger user"""
    url = "https://graph.facebook.com/v21.0/me/messages"
    params = {"access_token": PAGE_ACCESS_TOKEN}
    headers = {"Content-Type": "application/json"}
    data = {
        "recipient": {"id": recipient_id},
        "message": {"text": text}
    }
    response = requests.post(url, params=params, headers=headers, json=data)
    if response.status_code != 200:
        print(f"Error sending message: {response.text}")

In [9]:
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
